## Business Scenario

You are a data analyst and have a large ecommerce company in India (let’s call it X) as a client.
X gets a few thousand orders via their website on a daily basis and they have to deliver them as
fast as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India who charge them some amount per delivery.
The charges are dependent upon two factors:
● Weight of the product
● Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their partners per Order are correct.

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np

In [2]:
#load datasets
df_Order = pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Order Report.xlsx')
df_pin = pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - Pincode Zones.xlsx')
df_sku = pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Company X - SKU Master.xlsx')

dfc_Invoice = pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Invoice.xlsx')
dfc_rate= pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Courier Company - Rates.xlsx')
dfc_result = pd.read_excel('/content/drive/MyDrive/Cointab Data Analyst - Challenge/Expected_Result.xlsx')

In [3]:
#merge company x data
df = df_Order.merge(df_sku,on='SKU')
#calculate total wt of X data
df['Total weight'] = df['Order Qty']* df['Weight (g)']
#rename columns
df = df.rename(columns={"ExternOrderNo": "Order ID","Weight (g)": "Weight_slab_X", "Total weight": "Total_weight_X" })
df.head()

,Order ID,SKU,Order Qty,Weight_slab_X,Total_weight_X
0,2001827036,8904223818706,1.0,127,127.0
1,2001821995,8904223818706,1.0,127,127.0
2,2001819252,8904223818706,1.0,127,127.0
3,2001816996,8904223818706,1.0,127,127.0
4,2001814580,8904223818706,1.0,127,127.0


In [4]:
df.describe()

,Order ID,Order Qty,Weight_slab_X,Total_weight_X
count,4.010000e+02,401.000000,401.000000,401.000000
mean,2.001811e+09,1.296758,181.566085,220.638404
std,5.244070e+03,0.918811,92.583321,140.609891
min,2.001806e+09,1.000000,10.000000,10.000000
25%,2.001807e+09,1.000000,121.000000,127.000000
50%,2.001809e+09,1.000000,133.000000,210.000000
75%,2.001812e+09,1.000000,240.000000,240.000000
max,2.001827e+09,8.000000,600.000000,1104.000000


In [5]:
#merge courier invoice and X company data
df_M = dfc_Invoice.merge(df,on='Order ID')
df_M.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Order Qty,Weight_slab_X,Total_weight_X
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,2.0,240,480.0
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,6.0,137,822.0
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,1.0,127,127.0
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,1.0,240,240.0
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,1.0,133,133.0


In [6]:
#clean pin_x company X data and merge
df_pin = df_pin.drop(['Warehouse Pincode'],axis = 1)
df_pin = df_pin.rename(columns ={"Zone":"Zone_X"})
df_M = df_M.merge(df_pin,on='Customer Pincode')
df_M = df_M.drop(['Warehouse Pincode'], axis=1)

In [7]:
# create a list of our conditions
conditions = [
    (df_M['Total_weight_X'] >= 10) & (df_M['Total_weight_X'] <= 500),
    (df_M['Total_weight_X'] > 500) & (df_M['Total_weight_X'] <= 1000),
    (df_M['Total_weight_X'] > 1000) & (df_M['Total_weight_X'] <= 1500)
    ]

# create a list of the values we want to assign for each condition
values = ['0.5', '1', '1.5']

# create a new column and use np.select to assign values to it using our lists as arguments
df_M['slab'] = np.select(conditions, values)

# display updated DataFrame
df_M.head()


,AWB Code,Order ID,Charged Weight,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Order Qty,Weight_slab_X,Total_weight_X,Zone_X,slab
0,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,8904223819147,2.0,240,480.0,d,0.5
1,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,8904223818645,6.0,137,822.0,d,1
2,1091117222194,2001806273,1.0,486886,d,Forward charges,90.2,8904223818706,1.0,127,127.0,d,0.5
3,1091117222194,2001806273,1.0,486886,d,Forward charges,90.2,8904223818850,1.0,240,240.0,d,0.5
4,1091117222194,2001806273,1.0,486886,d,Forward charges,90.2,8904223818942,1.0,133,133.0,d,0.5


In [8]:
#convert slab to numeric
df_M['slab']= pd.to_numeric(df_M['slab'])

If the invoice mentions
“Forward charges” then only forward charges (“fwd”) should be applicable as per zone
and fixed & additional weights based on weight slabs. If the invoice mentions “Forward
and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be
applicable as per zone and fixed & additional weights based on weight slabs.

For the first 0.5 KG, “fixed” rate as per the slab is applicable. For each additional 0.5 KG,
“additional” weight in the same proportion is applicable. Total charges will be “fixed” +
“total additional” if any

In [9]:
conditions = [
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 1.5),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 1.5),
    (df_M['Type of Shipment'] == 'Forward charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 1.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 0.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 1),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'b') & (df_M['slab'] == 1.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'd') & (df_M['slab'] == 1.5),
    (df_M['Type of Shipment'] == 'Forward and RTO charges') & (df_M['Zone_X'] == 'e') & (df_M['slab'] == 1.5)]
    



# create a list of the values we want to assign for each condition
values = ['33', '45.4', '56.6','61.3', '90.2', '112.1', '89.6', '135', '112.1', '20.5', '41.3', '50.7', '48.8', '86.1', '106.2', '77.1', '130.9', '161.7']   

# create a new column and use np.select to assign values to it using our lists as arguments
df_M['Charges_X'] = np.select(conditions, values)

In [10]:
#convert gm to KG
df_M['Total weight as per X (KG)'] = df_M['Total_weight_X']/1000
df_M['Weight slab as per X (KG)'] = df_M['Weight_slab_X']/1000

In [11]:
#rename columns as per instructions
df_M = df_M.rename(columns={"Charged Weight":"Weight slab charged by Courier Company (KG)", "Zone":"Delivery Zone charged by Courier Company", "Zone_X":"Delivery Zone as per X", "Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)", "Charges_X":"Expected Charge as per X (Rs.)", "AWB Code":"AWB Number"})

In [12]:
#Total weight courier company
df_M['Total weight as per Courier Company (KG)'] = df_M['Order Qty']* df_M['Weight slab charged by Courier Company (KG)']

In [13]:
#select only required columns
df_New = df_M[["Order ID",  "AWB Number", "Total weight as per X (KG)", "Weight slab as per X (KG)", "Total weight as per Courier Company (KG)", "Weight slab charged by Courier Company (KG)", "Delivery Zone as per X", "Delivery Zone charged by Courier Company", "Expected Charge as per X (Rs.)", "Charges Billed by Courier Company (Rs.)"]]

In [14]:
df_New.head()

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.)
0,2001806232,1091117222124,0.480,0.240,2.6,1.3,d,d,45.4,135.0
1,2001806232,1091117222124,0.822,0.137,7.8,1.3,d,d,90.2,135.0
2,2001806273,1091117222194,0.127,0.127,1.0,1.0,d,d,45.4,90.2
3,2001806273,1091117222194,0.240,0.240,1.0,1.0,d,d,45.4,90.2
4,2001806273,1091117222194,0.133,0.133,1.0,1.0,d,d,45.4,90.2


In [15]:
#convert to neumeric
df_New['Expected Charge as per X (Rs.)']= pd.to_numeric(df_New['Expected Charge as per X (Rs.)'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#create a column diff of expected by company X and charges billed
df_New['Difference Between Expected Charges and Billed Charges (Rs.)'] = df_New['Expected Charge as per X (Rs.)'] - df_New['Charges Billed by Courier Company (Rs.)'] 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
#convert to string and upper
df_New['Delivery Zone as per X'] = df_New['Delivery Zone as per X'].str.upper()
df_New['Delivery Zone charged by Courier Company'] = df_New['Delivery Zone charged by Courier Company'].str.upper()

#df_New['Delivery Zone as per X'] = map(lambda x: x.upper(), df_New['Delivery Zone as per X'])
#df_New['Delivery Zone charged by Courier Company'] = map(lambda x: x.upper(), df_New['Delivery Zone charged by Courier Company'])
df_New.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,0.480,0.240,2.6,1.3,D,D,45.4,135.0,-89.6
1,2001806232,1091117222124,0.822,0.137,7.8,1.3,D,D,90.2,135.0,-44.8
2,2001806273,1091117222194,0.127,0.127,1.0,1.0,D,D,45.4,90.2,-44.8
3,2001806273,1091117222194,0.240,0.240,1.0,1.0,D,D,45.4,90.2,-44.8
4,2001806273,1091117222194,0.133,0.133,1.0,1.0,D,D,45.4,90.2,-44.8


In [18]:
#Save to csv the result
df_New.to_csv('Result.csv', index=False)

Create 3 dataframes

df_OO = same charge as expected by company X

df_OC = over charged by courier

df_UC = under charged by courier

In [19]:
df_OO = df_New[df_New['Difference Between Expected Charges and Billed Charges (Rs.)']==0]
df_OC = df_New[df_New['Difference Between Expected Charges and Billed Charges (Rs.)']<0]
df_UC = df_New[df_New['Difference Between Expected Charges and Billed Charges (Rs.)']>0]

In [20]:
df_OC['Order ID'].nunique()

116

In [21]:
df_OC['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

-47140.7

In [22]:
df_OO['Order ID'].nunique()

6

In [23]:
df_OO['Expected Charge as per X (Rs.)'].sum()

606.5999999999999

In [24]:
df_UC['Order ID'].nunique()

2

In [25]:
df_UC['Difference Between Expected Charges and Billed Charges (Rs.)'].sum()

259.6

In [26]:
dfc_Invoice['Order ID'].nunique()

124